In [0]:
from pyspark.sql.functions import *

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("AccountsDF").getOrCreate()

# Sample data
data = [
    (3, 108939),
    (2, 12747),
    (8, 87709),
    (6, 91796)
]

# Column names
columns = ["account_id", "income"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()


In [0]:
with cte as(
select 'Low Salary' as category,
sum(case when income <20000 then 1 else 0 end) as accounts_count
from accounts
union all
select 'Average Salary' as category,
sum(case when income between 20000 and 50000 then 1 else 0 end) as accounts_count
from accounts
union all
select 'High Salary' as category,
sum(case when income >50000 then 1 else 0 end) as accounts_count
from accounts
)
select * from cte
order by accounts_count desc



In [0]:
accounts_df = df.withColumn('category', when(col('income')<20000, 'Low Salary') \
    .when((col('income')>=20000) & (col('income')<=50000), 'Average Salary')    \
        .when(col('income')>50000, 'High Salary')
        )

category_counts_df = accounts_df.groupBy('category').agg(count('*').alias('accounts_count'))

all_categories = spark.createDataFrame([
    ("Low Salary",),
    ("Average Salary",),
    ("High Salary",)
], ["category"])
result_df = all_categories.join(category_counts_df, on='category', how='left').fillna(0)
result_df.display()